# National League Accumulator Simulation

This notebook demonstrates how to combine multiple matches into a betting accumulator and evaluate the expected return on investment (ROI).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import factorial


In [ ]:
# Load match data for several seasons
files = {
    '2023-24': 'england-national-league-matches-2023-to-2024-stats.csv',
    '2024-25': 'england-national-league-matches-2024-to-2025-stats.csv',
    '2025-26': 'england-national-league-matches-2025-to-2026-stats.csv'
}
seasons = {season: pd.read_csv(path) for season, path in files.items()}


In [ ]:
from math import exp

def implied_probs(home_odds, draw_odds, away_odds):
    probs = np.array([1/home_odds, 1/draw_odds, 1/away_odds], dtype=float)
    return probs / probs.sum()

def p_double_chance(row, selection='1X'):
    home, draw, away = implied_probs(row['odds_ft_home_team_win'], row['odds_ft_draw'], row['odds_ft_away_team_win'])
    if selection == '1X':
        return home + draw
    if selection == '12':
        return home + away
    if selection == 'X2':
        return draw + away
    raise ValueError('selection must be 1X, 12, or X2')

def poisson_pmf(lmbda, k):
    return (lmbda**k * exp(-lmbda)) / factorial(k)

def p_goal_range(row, low=2, high=3):
    lmbda = row['Home Team Pre-Match xG'] + row['Away Team Pre-Match xG']
    return sum(poisson_pmf(lmbda, g) for g in range(low, high+1))

def accumulator_probability(probs):
    return float(np.prod(probs))

def accumulator_odds(odds):
    return float(np.prod(odds))


In [ ]:
# Example accumulator using double chance selections
season_df = seasons['2024-25']
slate = season_df.head(3)
legs = []
for _, row in slate.iterrows():
    prob = p_double_chance(row, '1X')
    odds = 1 / prob
    result = (row['home_team_goal_count'] >= row['away_team_goal_count'])
    legs.append({'match': f"{row['home_team_name']} vs {row['away_team_name']}", 'prob': prob, 'odds': odds, 'result': result})
acc_prob = accumulator_probability([leg['prob'] for leg in legs])
acc_odds = accumulator_odds([leg['odds'] for leg in legs])
expected_roi = acc_prob * acc_odds - 1
print('Accumulator probability:', acc_prob)
print('Combined odds:', acc_odds)
print('Expected ROI:', expected_roi)


In [ ]:
# Simulate bankroll trajectories across seasons

def simulate_season(df, stake=1.0, leg_count=3):
    bankroll = [100]
    for date, day in df.groupby('date_GMT'):
        slate = day.head(leg_count)
        if len(slate) < leg_count:
            continue
        probs, odds, results = [], [], []
        for _, row in slate.iterrows():
            prob = p_double_chance(row, '1X')
            probs.append(prob)
            odds.append(1/prob)
            results.append(row['home_team_goal_count'] >= row['away_team_goal_count'])
        acc_prob = accumulator_probability(probs)
        acc_odds = accumulator_odds(odds)
        win = all(results)
        bankroll.append(bankroll[-1] - stake + (stake*acc_odds if win else 0))
    return bankroll

trajectories = {season: simulate_season(df) for season, df in seasons.items()}
for season, bal in trajectories.items():
    plt.plot(bal, label=season)
plt.title('Bankroll Trajectories')
plt.xlabel('Bet Number')
plt.ylabel('Bankroll')
plt.legend()
plt.show()
